In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import os,errno
import numpy as np
import sys
import datetime as dt
import scipy.stats as st
import pandas as pd

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2024,1)
ivt_list_merra2=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
    weights=np.cos(np.deg2rad(ds.lat))
    ivt=ds['ivt'][0::2,:,:].weighted(weights).mean().values.tolist()
    ivt_list_merra2.append(ivt)
    
yrs=np.arange(1940,2024,1)
ivt_list_era5=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
    weights=np.cos(np.deg2rad(ds.lat))
    ivt=ds['ivt'].weighted(weights).mean().values.tolist()
    ivt_list_era5.append(ivt)
    
yrs=np.arange(1958,2024,1)
ivt_list_jra55=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')
    weights=np.cos(np.deg2rad(ds.lat))
    ivt=ds['ivt'].weighted(weights).mean().values.tolist()
    ivt_list_jra55.append(ivt)

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)

yrs=np.arange(1980,2024,1)
test=[x for x in ivt_list_merra2]
r1=st.linregress(yrs,test)
change=r1[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r1[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in ivt_list_jra55[22:]]
r2=st.linregress(yrs,test)
change=r2[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='JRA-55 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r2[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in ivt_list_era5[40:]]
r3=st.linregress(yrs,test)
change=r3[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='ERA5 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r3[3],3))+')'

ax.plot(np.arange(1980,2024,1),ivt_list_merra2,linewidth=7,label=label_1)#,color='b')
ax.plot(np.arange(1958,2024,1),ivt_list_jra55,linewidth=7,label=label_2)#,color='orange')
ax.plot(np.arange(1940,2024,1),ivt_list_era5,linewidth=7,label=label_3)#,color='forestgreen')

#yrs=np.arange(1980,2024,1)
#ax.plot(yrs,[r1[0]*x+r1[1] for x in yrs],'k',linewidth=7,linestyle='--')
#ax.plot(yrs,[r2[0]*x+r2[1] for x in yrs],'k',linewidth=7,linestyle='--')
#ax.plot(yrs,[r3[0]*x+r3[1] for x in yrs],'k',linewidth=7,linestyle='--')

ax.grid(True)
ax.tick_params(labelsize=(24))
ax.set_xlabel('Year',fontsize=27)
ax.set_ylabel('kg m$^{-1}$ s$^{-1}$',fontsize=27)
ax.set_title(r'$\bf{Mean}$'+' '+r'$\bf{IVT}$',fontsize=40)
ax.set_ylim(153,196)
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.legend(loc='upper left',fontsize=23)
plt.show()

In [ ]:
fig.savefig(dir_data+'paper_ivt_lines.png')

In [ ]:
yrs=np.arange(1980,2023,1)
slopes_list_merra2=[]
sigs_list_merra2=[]
for i in range(len(yrs)):
    r=st.linregress(yrs[i:],ivt_list_merra2[i:])
    slopes_list_merra2.append(r[0])
    sigs_list_merra2.append(r[3])
    
yrs=np.arange(1940,2023,1)
slopes_list_era5=[]
sigs_list_era5=[]
for i in range(len(yrs)):
    r=st.linregress(yrs[i:],ivt_list_era5[i:])
    slopes_list_era5.append(r[0])
    sigs_list_era5.append(r[3])
    
yrs=np.arange(1958,2023,1)
slopes_list_jra55=[]
sigs_list_jra55=[]
for i in range(len(yrs)):
    r=st.linregress(yrs[i:],ivt_list_jra55[i:-1])
    slopes_list_jra55.append(r[0])
    sigs_list_jra55.append(r[3])

In [ ]:
a=0.05#/sum_factor

p_values=sigs_list_merra2+sigs_list_era5+sigs_list_jra55
p_values=[x for x in p_values if 0<x<=1]
N=len(p_values)
p_values=sorted(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)

In [ ]:
fig=plt.figure(figsize=(15,6))
ax=plt.subplot(1,1,1)

mean_merra2=float(sum(ivt_list_merra2))/float(len(ivt_list_merra2))
mean_era5=float(sum(ivt_list_era5))/float(len(ivt_list_era5))
mean_jra55=float(sum(ivt_list_jra55))/float(len(ivt_list_jra55))

slopes_list_merra2_percent=[x/mean_merra2*100. for x in slopes_list_merra2]
slopes_list_era5_percent=[x/mean_era5*100. for x in slopes_list_era5]
slopes_list_jra55_percent=[x/mean_jra55*100. for x in slopes_list_jra55]

slopes_list_merra2_filter=[x if y<=p_final else np.nan for x,y in zip(slopes_list_merra2,sigs_list_merra2)]
slopes_list_era5_filter=[x if y<=p_final else np.nan for x,y in zip(slopes_list_era5,sigs_list_era5)]
slopes_list_jra55_filter=[x if y<=p_final else np.nan for x,y in zip(slopes_list_jra55,sigs_list_jra55)]

slopes_array=np.zeros((3,83))
slopes_array[0,-43:]=slopes_list_merra2_filter
slopes_array[1,-65:]=slopes_list_jra55_filter
slopes_array[2,:]=slopes_list_era5_filter

slopes_array[:,-5:]=np.nan

cax=ax.pcolormesh(np.arange(1940,2023,1),np.arange(3),slopes_array,vmin=-0.5,vmax=0.5,cmap=plt.cm.BrBG)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both')
ax.set_title(r'$\bf{Significant}$'+' '+r'$\bf{IVT}$'+' '+r'$\bf{trends}$'+' '+r'$\bf{X-2022}$',fontsize=35)
ax.tick_params(labelsize=20)
ax.set_xlabel('Year',fontsize=22)
plt.yticks([0,1,2])
ax.set_yticklabels(['MERRA-2','JRA-55','ERA5'],rotation=90,va='center')
cbar.ax.tick_params(labelsize=20)
cbar.set_label('% yr$^{-1}$',fontsize=22)

plt.show()

In [ ]:
fig.savefig(dir_data+'paper_ivt_sigs.png')

In [ ]:
yrs=np.arange(1980,2024,1)
ivt_list_merra2=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
    weights=np.cos(np.deg2rad(ds.lat))
    ivt=ds['iwv'][0::2,:,:].weighted(weights).mean().values.tolist()
    ivt_list_merra2.append(ivt)
    
yrs=np.arange(1940,2024,1)
ivt_list_era5=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
    weights=np.cos(np.deg2rad(ds.lat))
    ivt=ds['iwv'].weighted(weights).mean().values.tolist()
    ivt_list_era5.append(ivt)
    
yrs=np.arange(1958,2024,1)
ivt_list_jra55=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir3+'jra55_iwv_'+str(year)+'.nc')
    weights=np.cos(np.deg2rad(ds.lat))
    ivt=ds['iwv'].weighted(weights).mean().values.tolist()
    ivt_list_jra55.append(ivt)

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)

yrs=np.arange(1980,2024,1)
test=[x for x in ivt_list_merra2]
r1=st.linregress(yrs,test)
change=r1[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r1[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in ivt_list_jra55[22:]]
r2=st.linregress(yrs,test)
change=r2[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='JRA-55 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r2[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in ivt_list_era5[40:]]
r3=st.linregress(yrs,test)
change=r3[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='ERA5 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r3[3],3))+')'

ax.plot(np.arange(1980,2024,1),ivt_list_merra2,linewidth=7,label=label_1)#,color='b')
ax.plot(np.arange(1958,2024,1),ivt_list_jra55,linewidth=7,label=label_2)#,color='orange')
ax.plot(np.arange(1940,2024,1),ivt_list_era5,linewidth=7,label=label_3)#,color='forestgreen')
ax.grid(True)
ax.tick_params(labelsize=(24))
ax.set_xlabel('Year',fontsize=27)
ax.set_ylabel('kg m$^{-2}$',fontsize=27)
ax.set_title(r'$\bf{Mean}$'+' '+r'$\bf{IWV}$',fontsize=40)
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.legend(loc='upper left',fontsize=23)
ax.set_ylim(22.5,26.7)
plt.show()

In [ ]:
fig.savefig(dir_data+'paper_iwv_lines.png')

In [ ]:
yrs=np.arange(1980,2023,1)
slopes_list_merra2=[]
sigs_list_merra2=[]
for i in range(len(yrs)):
    r=st.linregress(yrs[i:],ivt_list_merra2[i:])
    slopes_list_merra2.append(r[0])
    sigs_list_merra2.append(r[3])
    
yrs=np.arange(1940,2023,1)
slopes_list_era5=[]
sigs_list_era5=[]
for i in range(len(yrs)):
    r=st.linregress(yrs[i:],ivt_list_era5[i:])
    slopes_list_era5.append(r[0])
    sigs_list_era5.append(r[3])
    
yrs=np.arange(1958,2023,1)
slopes_list_jra55=[]
sigs_list_jra55=[]
for i in range(len(yrs)):
    r=st.linregress(yrs[i:],ivt_list_jra55[i:-1])
    slopes_list_jra55.append(r[0])
    sigs_list_jra55.append(r[3])

In [ ]:
a=0.05#/sum_factor

p_values=sigs_list_merra2+sigs_list_era5+sigs_list_jra55
p_values=[x for x in p_values if 0<x<=1]
N=len(p_values)
p_values=sorted(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)

In [ ]:
fig=plt.figure(figsize=(15,6))
ax=plt.subplot(1,1,1)

mean_merra2=float(sum(ivt_list_merra2))/float(len(ivt_list_merra2))
mean_era5=float(sum(ivt_list_era5))/float(len(ivt_list_era5))
mean_jra55=float(sum(ivt_list_jra55))/float(len(ivt_list_jra55))

slopes_list_merra2_percent=[x/mean_merra2*100. for x in slopes_list_merra2]
slopes_list_era5_percent=[x/mean_era5*100. for x in slopes_list_era5]
slopes_list_jra55_percent=[x/mean_jra55*100. for x in slopes_list_jra55]

slopes_list_merra2_filter=[x if y<=p_final else np.nan for x,y in zip(slopes_list_merra2,sigs_list_merra2)]
slopes_list_era5_filter=[x if y<=p_final else np.nan for x,y in zip(slopes_list_era5,sigs_list_era5)]
slopes_list_jra55_filter=[x if y<=p_final else np.nan for x,y in zip(slopes_list_jra55,sigs_list_jra55)]

slopes_array=np.zeros((3,83))
slopes_array[0,-43:]=slopes_list_merra2_filter
slopes_array[1,-65:]=slopes_list_jra55_filter
slopes_array[2,:]=slopes_list_era5_filter

slopes_array[:,-5:]=np.nan

cax=ax.pcolormesh(np.arange(1940,2023,1),np.arange(3),slopes_array,vmin=-0.1,vmax=0.1,cmap=plt.cm.BrBG)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both')
ax.set_title(r'$\bf{Significant}$'+' '+r'$\bf{IWV}$'+' '+r'$\bf{trends}$'+' '+r'$\bf{X-2022}$',fontsize=35)
ax.tick_params(labelsize=20)
ax.set_xlabel('Year',fontsize=22)
plt.yticks([0,1,2])
ax.set_yticklabels(['MERRA-2','JRA-55','ERA5'],rotation=90,va='center')
cbar.ax.tick_params(labelsize=20)
cbar.set_label('% yr$^{-1}$',fontsize=22)

plt.show()

In [ ]:
fig.savefig(dir_data+'paper_iwv_sigs.png')

In [ ]:
yrs=np.arange(1980,2024,1)
s_list_merra2=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
        
    weights=np.cos(np.deg2rad(ds.lat))
    s=s.weighted(weights).mean().values.tolist()
    s_list_merra2.append(s)
    
yrs=np.arange(1940,2024,1)
s_list_era5=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
    
    weights=np.cos(np.deg2rad(ds.lat))
    s=s.weighted(weights).mean().values.tolist()
    s_list_era5.append(s)
    
yrs=np.arange(1958,2024,1)
s_list_jra55=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
    
    weights=np.cos(np.deg2rad(ds.lat))
    s=s.weighted(weights).mean().values.tolist()
    s_list_jra55.append(s)

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)

yrs=np.arange(1980,2024,1)
test=[x for x in s_list_merra2]
r1=st.linregress(yrs,test)
change=r1[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r1[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in s_list_jra55[22:]]
r2=st.linregress(yrs,test)
change=r2[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='JRA-55 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r2[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in s_list_era5[40:]]
r3=st.linregress(yrs,test)
change=r3[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='ERA5 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r3[3],3))+')'

ax.plot(np.arange(1980,2024,1),s_list_merra2,linewidth=7,label=label_1)#,color='b')
ax.plot(np.arange(1958,2024,1),s_list_jra55,linewidth=7,label=label_2)#,color='orange')
ax.plot(np.arange(1940,2024,1),s_list_era5,linewidth=7,label=label_3)#,color='forestgreen')
ax.grid(True)
ax.tick_params(labelsize=(24))
ax.set_xlabel('Year',fontsize=27)
ax.set_ylabel('m s$^{-1}$',fontsize=27)
ax.set_title(r'$\bf{Mean}$'+' '+r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{wind}$'+' '+r'$\bf{speed}$',fontsize=40)
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.legend(loc='upper left',fontsize=23)
ax.set_ylim(7.8,9.3)
plt.show()

In [ ]:
fig.savefig(dir_data+'paper_s_lines.png')

In [ ]:
yrs=np.arange(1980,2024,1)
s1_list_merra2=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
    v=ds['v']
    
    for j in range(1,13,1):
        times=[x for x in ds.time.values if pd.to_datetime(x).month==j]
        u_sel=u.sel(time=times)
        v_sel=v.sel(time=times)
    
        u_mean=u_sel.mean(dim='time')
        v_mean=v_sel.mean(dim='time')

        s_mean=np.sqrt(u_mean**2+v_mean**2)

        weights=np.cos(np.deg2rad(ds.lat))
        s_mean=s_mean.weighted(weights).mean(skipna=True).values.tolist()
        s1_list_merra2.append(s_mean)
    
yrs=np.arange(1940,2024,1)
s1_list_era5=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
    v=ds['v']
    
    for j in range(1,13,1):
        times=[x for x in ds.time.values if pd.to_datetime(x).month==j]
        u_sel=u.sel(time=times)
        v_sel=v.sel(time=times)
    
        u_mean=u_sel.mean(dim='time')
        v_mean=v_sel.mean(dim='time')

        s_mean=np.sqrt(u_mean**2+v_mean**2)

        weights=np.cos(np.deg2rad(ds.lat))
        s_mean=s_mean.weighted(weights).mean(skipna=True).values.tolist()
        s1_list_era5.append(s_mean)
    
yrs=np.arange(1958,2024,1)
s1_list_jra55=[]
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
    v=ds['v']
    
    for j in range(1,13,1):
        times=[x for x in ds.time.values if pd.to_datetime(x).month==j]
        u_sel=u.sel(time=times)
        v_sel=v.sel(time=times)
    
        u_mean=u_sel.mean(dim='time')
        v_mean=v_sel.mean(dim='time')

        s_mean=np.sqrt(u_mean**2+v_mean**2)

        weights=np.cos(np.deg2rad(ds.lat))
        s_mean=s_mean.weighted(weights).mean(skipna=True).values.tolist()
        s1_list_jra55.append(s_mean)

In [ ]:
yrs=np.arange(1940,2024,1)
s1_list_era5_annual=[]
for i in range(len(yrs)):
    s1_sel=s1_list_era5[12*i:12*(i+1)]
    s1_mean=float(sum(s1_sel))/float(len(s1_sel))
    s1_list_era5_annual.append(s1_mean)
    
yrs=np.arange(1980,2024,1)
s1_list_merra2_annual=[]
for i in range(len(yrs)):
    s1_sel=s1_list_merra2[12*i:12*(i+1)]
    s1_mean=float(sum(s1_sel))/float(len(s1_sel))
    s1_list_merra2_annual.append(s1_mean)
    
yrs=np.arange(1958,2024,1)
s1_list_jra55_annual=[]
for i in range(len(yrs)):
    s1_sel=s1_list_jra55[12*i:12*(i+1)]
    s1_mean=float(sum(s1_sel))/float(len(s1_sel))
    s1_list_jra55_annual.append(s1_mean)


fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)

yrs=np.arange(1980,2024,1)
test=[x for x in s1_list_merra2_annual]
r1=st.linregress(yrs,test)
change=r1[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r1[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in s1_list_jra55_annual[22:]]
r2=st.linregress(yrs,test)
change=r2[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='JRA-55 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r2[3],3))+')'

yrs=np.arange(1980,2024,1)
test=[x for x in s1_list_era5_annual[40:]]
r3=st.linregress(yrs,test)
change=r3[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='ERA5 (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r3[3],3))+')'

ax.plot(np.arange(1980,2024,1),s1_list_merra2_annual,linewidth=7,label=label_1)#,color='b')
ax.plot(np.arange(1958,2024,1),s1_list_jra55_annual,linewidth=7,label=label_2)#,color='orange')
ax.plot(np.arange(1940,2024,1),s1_list_era5_annual,linewidth=7,label=label_3)#,color='forestgreen')
ax.grid(True)
ax.tick_params(labelsize=(24))
ax.set_xlabel('Year',fontsize=27)
ax.set_ylabel('m s$^{-1}$',fontsize=27)
ax.set_title(r'$\bf{Monthly}$'+' '+r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{circulation}$',fontsize=40)
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.legend(loc='upper left',fontsize=23)
ax.set_ylim(5.1,6.5)
plt.show()

In [ ]:
#fig.savefig(dir_data+'paper_s1_lines.png')
fig.savefig(dir_data+'AR_PAPER_NEW_2.png')